In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import sqlalchemy
from sqlalchemy import create_engine, func, inspect, text

In [2]:
df = pd.read_csv("merged_df.csv")
df.head()


,latitude,longitude,Country,Website,Studio,Publisher,PC,Mobile,VR,VR only,Console,Browser games
0,61.924110,25.748151,Finland,10tons.com/,10tons,?,X,X,?,?,?,?
1,51.919438,19.145136,Poland,11bitstudios.com/,11 bit studios,11 bit studios,X,X,?,?,X,?
2,64.963051,-19.020835,Iceland,1939games.com/,1939 Games,?,X,?,?,?,?,?
3,46.227638,2.213749,France,1button.co/,1Button,?,?,X,?,?,?,?
4,61.524010,105.318756,Russia,1c.games/ru/,1C Games,1C,X,?,?,?,X,?


In [3]:
df2 = pd.read_csv("games.csv")
df2.head()

,Title,Release Date,Publisher,Sub Publisher,Rating,Times Listed,Number of Reviews,Genres,Sub Genre,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,Return of the Obra Dinn,18-Oct-18,3909,Lucas Pope,4.3,1K,1K,Adventure,Indie,"In this 1-bit first-person mystery game, a mer...",['This game made me feel so god damn smart. Ju...,4.7K,195,2.6K,2.4K
1,Deltarune: Chapter 2,17-Sep-21,4-Aug,tobyfox,4.3,737,737,Adventure,Indie,Deltarune: Chapter 2 is the next installment o...,"['Um jogo espetacular, amo as piadas e a gamep...",7.5K,222,2.1K,753
2,Deltarune: Chapter 1,31-Oct-18,4-Aug,tobyfox,3.9,591,591,Adventure,Indie,The story follows a human named Kris who lives...,"['√ìtima apresenta√ß√£o a um novo universo, no...",11K,107,1.8K,729
3,Umineko: When They Cry Chiru,15-Sep-09,07th Expansion,NaN,4.6,324,324,Adventure,Visual Novel,Umineko no Naku Koro ni Chiru is the second ha...,"['cried like a little bitch ngl', ""God I reall...",1.7K,108,582,493
4,Higurashi: When They Cry,10-Aug-02,07th Expansion,MangaGamer,4.2,311,311,Adventure,Visual Novel,Higurashi: When They Cry is a Japanese murder ...,['umidrones/umiroaches/uminekoids will rant an...,1.1K,97,545,435


In [4]:
engine = create_engine("sqlite:///video_games_final.sqlite")

df.to_sql("merged_df", con=engine, if_exists="replace", index=False)
# df2.to_sql("games", con=engine, if_exists="replace", index=False)


600

In [5]:
df2.dropna(subset=["Rating"], inplace=True)

In [6]:
# Experimental way to convert K
# Define the columns to be converted
columns_to_convert = ["Times Listed", "Number of Reviews", "Plays", "Playing", "Backlogs", "Wishlist"]

# Iterate over the columns and convert the values
for column in columns_to_convert:
    # Remove the "k" and convert to thousands
    df2[column] = df2[column].apply(lambda x: float(x[:-1]) * 1000 if x.endswith("K") else x)
    
    df2[column] = pd.to_numeric(df2[column])

# Print the updated DataFrame
print(df2)

                             Title Release Date       Publisher  \
0          Return of the Obra Dinn    18-Oct-18            3909   
1             Deltarune: Chapter 2    17-Sep-21           4-Aug   
2             Deltarune: Chapter 1    31-Oct-18           4-Aug   
3     Umineko: When They Cry Chiru    15-Sep-09  07th Expansion   
4         Higurashi: When They Cry    10-Aug-02  07th Expansion   
...                            ...          ...             ...   
1125                 Disco Elysium    15-Oct-19           ZA/UM   
1126                     Fallout 3    28-Oct-08   ZeniMax Media   
1127               Life is Strange    29-Jan-15             NaN   
1128             Life is Strange 2    26-Sep-18             NaN   
1129          NEET Girl Date Night    21-Oct-22             NaN   

              Sub Publisher  Rating  Times Listed  Number of Reviews  \
0                Lucas Pope     4.3        1000.0             1000.0   
1                   tobyfox     4.3         737.0  

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1117 entries, 0 to 1129
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              1117 non-null   object 
 1   Release Date       1117 non-null   object 
 2   Publisher          1114 non-null   object 
 3   Sub Publisher      872 non-null    object 
 4   Rating             1117 non-null   float64
 5   Times Listed       1117 non-null   float64
 6   Number of Reviews  1117 non-null   float64
 7   Genres             1115 non-null   object 
 8   Sub Genre          886 non-null    object 
 9   Summary            1117 non-null   object 
 10  Reviews            1117 non-null   object 
 11  Plays              1117 non-null   float64
 12  Playing            1117 non-null   float64
 13  Backlogs           1117 non-null   float64
 14  Wishlist           1117 non-null   float64
dtypes: float64(7), object(8)
memory usage: 139.6+ KB


In [8]:
df2.to_sql("games", con=engine, if_exists="replace", index=False)

1117

In [9]:
# explore and understand the data

# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Using the inspector to print the column names within each table and its types
for table in tables:
    print(table)
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])
        
    print()

games
Title TEXT
Release Date TEXT
Publisher TEXT
Sub Publisher TEXT
Rating FLOAT
Times Listed FLOAT
Number of Reviews FLOAT
Genres TEXT
Sub Genre TEXT
Summary TEXT
Reviews TEXT
Plays FLOAT
Playing FLOAT
Backlogs FLOAT
Wishlist FLOAT

merged_df
latitude FLOAT
longitude FLOAT
Country TEXT
Website TEXT
Studio TEXT
Publisher TEXT
PC TEXT
Mobile TEXT
VR TEXT
VR only TEXT
Console TEXT
Browser games TEXT



In [10]:
df['Country'].nunique()

38

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   latitude       481 non-null    float64
 1   longitude      481 non-null    float64
 2   Country        600 non-null    object 
 3   Website        600 non-null    object 
 4   Studio         600 non-null    object 
 5   Publisher      600 non-null    object 
 6   PC             600 non-null    object 
 7   Mobile         600 non-null    object 
 8   VR             600 non-null    object 
 9   VR only        600 non-null    object 
 10  Console        600 non-null    object 
 11  Browser games  600 non-null    object 
dtypes: float64(2), object(10)
memory usage: 56.4+ KB


In [12]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1117 entries, 0 to 1129
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              1117 non-null   object 
 1   Release Date       1117 non-null   object 
 2   Publisher          1114 non-null   object 
 3   Sub Publisher      872 non-null    object 
 4   Rating             1117 non-null   float64
 5   Times Listed       1117 non-null   float64
 6   Number of Reviews  1117 non-null   float64
 7   Genres             1115 non-null   object 
 8   Sub Genre          886 non-null    object 
 9   Summary            1117 non-null   object 
 10  Reviews            1117 non-null   object 
 11  Plays              1117 non-null   float64
 12  Playing            1117 non-null   float64
 13  Backlogs           1117 non-null   float64
 14  Wishlist           1117 non-null   float64
dtypes: float64(7), object(8)
memory usage: 139.6+ KB


In [13]:
Country = "Russia"

# allow the user to select ALL or a specific state
if Country == "All":
    where_clause = "1=1"
else:
    where_clause = f"Country = '{Country}'"

query = f"""
        SELECT
            *
        FROM
            merged_df
        WHERE
            {where_clause};
"""

df_map = pd.read_sql(text(query), con=engine)
df_map.head()

,latitude,longitude,Country,Website,Studio,Publisher,PC,Mobile,VR,VR only,Console,Browser games
0,61.52401,105.318756,Russia,1c.games/ru/,1C Games,1C,X,?,?,?,X,?
1,61.52401,105.318756,Russia,2zombie.com/,2 Zombie Games,?,X,X,?,?,X,?
2,61.52401,105.318756,Russia,absolutsoft.com/,AbsolutSoft,?,X,?,?,?,?,?
3,61.52401,105.318756,Russia,adrovgames.com/,AdroVGames,?,X,?,?,?,?,?
4,61.52401,105.318756,Russia,aksweb.ru/,AK Studio,?,X,X,?,?,?,?


In [14]:
Country = "Finland"

# allow the user to select ALL or a specific state
if Country == "All":
    where_clause = "1=1"
else:
    where_clause = f"Country = '{Country}'"

query = f"""
        SELECT
            *
        FROM
            merged_df
        WHERE
            {where_clause};
"""

print(query)


        SELECT
            *
        FROM
            merged_df
        WHERE
            Country = 'Finland';



In [15]:
df_map = pd.read_sql(text(query), con=engine)
df_map.head()

,latitude,longitude,Country,Website,Studio,Publisher,PC,Mobile,VR,VR only,Console,Browser games
0,61.92411,25.748151,Finland,10tons.com/,10tons,?,X,X,?,?,?,?
1,61.92411,25.748151,Finland,3rdeyestudios.fi/#home-section,3rd eye studios,?,X,?,X,?,?,?
2,61.92411,25.748151,Finland,actionreaction.games/,Action reaction games,?,?,X,?,?,?,?
3,61.92411,25.748151,Finland,actionsquadstudios.com/,Action Squad Studios,Daedalic Entertainment,X,?,?,?,?,?
4,61.92411,25.748151,Finland,actnormalgames.com/,Act Normal Games,?,X,?,?,?,?,?


In [16]:
df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   latitude       20 non-null     float64
 1   longitude      20 non-null     float64
 2   Country        20 non-null     object 
 3   Website        20 non-null     object 
 4   Studio         20 non-null     object 
 5   Publisher      20 non-null     object 
 6   PC             20 non-null     object 
 7   Mobile         20 non-null     object 
 8   VR             20 non-null     object 
 9   VR only        20 non-null     object 
 10  Console        20 non-null     object 
 11  Browser games  20 non-null     object 
dtypes: float64(2), object(10)
memory usage: 2.0+ KB


In [17]:
Country = "All"

# allow the user to select ALL or a specific state
if Country == "All":
    where_clause = "1=1"
else:
    where_clause = f"Country = '{Country}'"
    
query = f"""
    SELECT
        Studio,
        Country,
        count(*) as num_studios
    FROM
        merged_df
    WHERE
        {where_clause}
    GROUP BY
        Country
    ORDER BY
        num_studios desc;
"""

df_bar = pd.read_sql(text(query), con=engine)
df_bar.head()

,Studio,Country,num_studios
0,1CC Games,UK,115
1,1C Games,Russia,67
2,11 bit studios,Poland,54
3,Alpha Blend Interactive,Germany,53
4,1Button,France,45


In [18]:
Country = "All"

# allow the user to select ALL or a specific Country
if Country == "All":
    where_clause = "1=1"
else:
    where_clause = f"Country = '{Country}'"
    
country_query = f"""
    SELECT
        Country,
        count(*) as num_studios
    FROM
        merged_df
    WHERE
        {where_clause}
    GROUP BY
        Country
    ORDER BY
        num_studios desc
    LIMIT 10;
"""

df_bar2 = pd.read_sql(text(country_query), con=engine)
df_bar2.head()

,Country,num_studios
0,UK,115
1,Russia,67
2,Poland,54
3,Germany,53
4,France,45


In [19]:
Country = "All"

    # allow the user to select ALL or a specific Country
if Country == "All":
        where_clause = "1=1"
else:
        where_clause = f"Country = '{Country}'"

country_query2 = f"""
        SELECT
            Country,
            count(*) as num_studios
        FROM
            merged_df
        WHERE
            {where_clause}
        GROUP BY
            Country
        ORDER BY
            num_studios desc
        LIMIT 10;
    """

df_bar3 = pd.read_sql(text(country_query2), con=engine)
df_bar3.head()

,Country,num_studios
0,UK,115
1,Russia,67
2,Poland,54
3,Germany,53
4,France,45


In [20]:
df2.head()

,Title,Release Date,Publisher,Sub Publisher,Rating,Times Listed,Number of Reviews,Genres,Sub Genre,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,Return of the Obra Dinn,18-Oct-18,3909,Lucas Pope,4.3,1000.0,1000.0,Adventure,Indie,"In this 1-bit first-person mystery game, a mer...",['This game made me feel so god damn smart. Ju...,4700.0,195.0,2600.0,2400.0
1,Deltarune: Chapter 2,17-Sep-21,4-Aug,tobyfox,4.3,737.0,737.0,Adventure,Indie,Deltarune: Chapter 2 is the next installment o...,"['Um jogo espetacular, amo as piadas e a gamep...",7500.0,222.0,2100.0,753.0
2,Deltarune: Chapter 1,31-Oct-18,4-Aug,tobyfox,3.9,591.0,591.0,Adventure,Indie,The story follows a human named Kris who lives...,"['√ìtima apresenta√ß√£o a um novo universo, no...",11000.0,107.0,1800.0,729.0
3,Umineko: When They Cry Chiru,15-Sep-09,07th Expansion,NaN,4.6,324.0,324.0,Adventure,Visual Novel,Umineko no Naku Koro ni Chiru is the second ha...,"['cried like a little bitch ngl', ""God I reall...",1700.0,108.0,582.0,493.0
4,Higurashi: When They Cry,10-Aug-02,07th Expansion,MangaGamer,4.2,311.0,311.0,Adventure,Visual Novel,Higurashi: When They Cry is a Japanese murder ...,['umidrones/umiroaches/uminekoids will rant an...,1100.0,97.0,545.0,435.0


In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1117 entries, 0 to 1129
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              1117 non-null   object 
 1   Release Date       1117 non-null   object 
 2   Publisher          1114 non-null   object 
 3   Sub Publisher      872 non-null    object 
 4   Rating             1117 non-null   float64
 5   Times Listed       1117 non-null   float64
 6   Number of Reviews  1117 non-null   float64
 7   Genres             1115 non-null   object 
 8   Sub Genre          886 non-null    object 
 9   Summary            1117 non-null   object 
 10  Reviews            1117 non-null   object 
 11  Plays              1117 non-null   float64
 12  Playing            1117 non-null   float64
 13  Backlogs           1117 non-null   float64
 14  Wishlist           1117 non-null   float64
dtypes: float64(7), object(8)
memory usage: 139.6+ KB


In [22]:
Rating = "All"
if Rating == "All":
        where_clause2 = "Rating >=3.5"
else:
        where_clause2 = f"Rating = '{Rating}'"
        
# Top 10 rated
title_query2 = f"""
        SELECT
            Title,
            Rating
        FROM
            games
        WHERE
            {where_clause2}
        GROUP BY
            Title
        ORDER BY
            Rating desc
        LIMIT 10;
    """


df2_bar3 = pd.read_sql(text(title_query2), con=engine)
df2_bar3.head(11)

,Title,Rating
0,Elden Ring: Shadow of the Erdtree,4.8
1,Umineko: When They Cry Chiru,4.6
2,Tokyo Necro,4.6
3,The Great Ace Attorney 2: Resolve,4.6
4,Sekiro: Shadows Die Twice - GOTY Edition,4.6
5,Outer Wilds,4.6
6,Metal Gear Solid 3: Subsistence,4.6
7,Metal Gear Solid 3: Snake Eater HD Edition,4.6
8,Hitman World of Assassination,4.6
9,Half-Life: Alyx,4.6


In [23]:
if Rating == "All":
        where_clause3 = "Rating <=3.5"
else:
        where_clause3 = f"Rating = '{Rating}'"
        
# Lowest 10 rated
title_query3 = f"""
        SELECT
            Title,
            Rating
        FROM
            games
        WHERE
            {where_clause3}
        GROUP BY
            Title
        ORDER BY
            Rating asc
        LIMIT 10;
    """


df2_bar4 = pd.read_sql(text(title_query3), con=engine)
df2_bar4.head(11)

,Title,Rating
0,Superman: The New Superman Adventures,0.7
1,Garten of Banban,1.2
2,Mega Man X7,1.4
3,Garten of Banban 2,1.5
4,Devil May Cry 2,1.6
5,Mighty No. 9,1.6
6,Dragon Ball GT: Final Bout,1.8
7,Sonic Forces,1.9
8,Sonic and the Secret Rings,1.9
9,Slender: The Eight Pages,2.0


In [24]:
df2.head()

,Title,Release Date,Publisher,Sub Publisher,Rating,Times Listed,Number of Reviews,Genres,Sub Genre,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,Return of the Obra Dinn,18-Oct-18,3909,Lucas Pope,4.3,1000.0,1000.0,Adventure,Indie,"In this 1-bit first-person mystery game, a mer...",['This game made me feel so god damn smart. Ju...,4700.0,195.0,2600.0,2400.0
1,Deltarune: Chapter 2,17-Sep-21,4-Aug,tobyfox,4.3,737.0,737.0,Adventure,Indie,Deltarune: Chapter 2 is the next installment o...,"['Um jogo espetacular, amo as piadas e a gamep...",7500.0,222.0,2100.0,753.0
2,Deltarune: Chapter 1,31-Oct-18,4-Aug,tobyfox,3.9,591.0,591.0,Adventure,Indie,The story follows a human named Kris who lives...,"['√ìtima apresenta√ß√£o a um novo universo, no...",11000.0,107.0,1800.0,729.0
3,Umineko: When They Cry Chiru,15-Sep-09,07th Expansion,NaN,4.6,324.0,324.0,Adventure,Visual Novel,Umineko no Naku Koro ni Chiru is the second ha...,"['cried like a little bitch ngl', ""God I reall...",1700.0,108.0,582.0,493.0
4,Higurashi: When They Cry,10-Aug-02,07th Expansion,MangaGamer,4.2,311.0,311.0,Adventure,Visual Novel,Higurashi: When They Cry is a Japanese murder ...,['umidrones/umiroaches/uminekoids will rant an...,1100.0,97.0,545.0,435.0


In [25]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1117 entries, 0 to 1129
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              1117 non-null   object 
 1   Release Date       1117 non-null   object 
 2   Publisher          1114 non-null   object 
 3   Sub Publisher      872 non-null    object 
 4   Rating             1117 non-null   float64
 5   Times Listed       1117 non-null   float64
 6   Number of Reviews  1117 non-null   float64
 7   Genres             1115 non-null   object 
 8   Sub Genre          886 non-null    object 
 9   Summary            1117 non-null   object 
 10  Reviews            1117 non-null   object 
 11  Plays              1117 non-null   float64
 12  Playing            1117 non-null   float64
 13  Backlogs           1117 non-null   float64
 14  Wishlist           1117 non-null   float64
dtypes: float64(7), object(8)
memory usage: 139.6+ KB


In [26]:
df3 = df2.copy()

In [27]:
# # Define a function to convert 'k' suffix to integers
# col_to_convert = ["Times Listed", "Number of Reviews", "Plays", "Playing", "Backlogs", "Wishlist"]
# def convert_to_int(value):
#     if 'K' in value or 'k' in value:
#         return int(float(value[:-1]) * 1000)
#     else:
#         return int(value)
# # Apply the conversion function to the specified columns
# df2[col_to_convert] = df2[col_to_convert].apply(lambda x: x.apply(convert_to_int))
# df3 = df2.copy()

In [28]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1117 entries, 0 to 1129
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              1117 non-null   object 
 1   Release Date       1117 non-null   object 
 2   Publisher          1114 non-null   object 
 3   Sub Publisher      872 non-null    object 
 4   Rating             1117 non-null   float64
 5   Times Listed       1117 non-null   float64
 6   Number of Reviews  1117 non-null   float64
 7   Genres             1115 non-null   object 
 8   Sub Genre          886 non-null    object 
 9   Summary            1117 non-null   object 
 10  Reviews            1117 non-null   object 
 11  Plays              1117 non-null   float64
 12  Playing            1117 non-null   float64
 13  Backlogs           1117 non-null   float64
 14  Wishlist           1117 non-null   float64
dtypes: float64(7), object(8)
memory usage: 139.6+ KB


In [29]:
df2.head()

,Title,Release Date,Publisher,Sub Publisher,Rating,Times Listed,Number of Reviews,Genres,Sub Genre,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,Return of the Obra Dinn,18-Oct-18,3909,Lucas Pope,4.3,1000.0,1000.0,Adventure,Indie,"In this 1-bit first-person mystery game, a mer...",['This game made me feel so god damn smart. Ju...,4700.0,195.0,2600.0,2400.0
1,Deltarune: Chapter 2,17-Sep-21,4-Aug,tobyfox,4.3,737.0,737.0,Adventure,Indie,Deltarune: Chapter 2 is the next installment o...,"['Um jogo espetacular, amo as piadas e a gamep...",7500.0,222.0,2100.0,753.0
2,Deltarune: Chapter 1,31-Oct-18,4-Aug,tobyfox,3.9,591.0,591.0,Adventure,Indie,The story follows a human named Kris who lives...,"['√ìtima apresenta√ß√£o a um novo universo, no...",11000.0,107.0,1800.0,729.0
3,Umineko: When They Cry Chiru,15-Sep-09,07th Expansion,NaN,4.6,324.0,324.0,Adventure,Visual Novel,Umineko no Naku Koro ni Chiru is the second ha...,"['cried like a little bitch ngl', ""God I reall...",1700.0,108.0,582.0,493.0
4,Higurashi: When They Cry,10-Aug-02,07th Expansion,MangaGamer,4.2,311.0,311.0,Adventure,Visual Novel,Higurashi: When They Cry is a Japanese murder ...,['umidrones/umiroaches/uminekoids will rant an...,1100.0,97.0,545.0,435.0


In [30]:
df2['Plays'].head(50)

0      4700.0
1      7500.0
2     11000.0
3      1700.0
4      1100.0
5      4800.0
6      2400.0
7     17000.0
8      5400.0
9      9100.0
10      738.0
11     5700.0
12     2700.0
13     9100.0
14     7600.0
15     3500.0
16     5100.0
17     1200.0
18       59.0
19     3700.0
20     5400.0
21     9600.0
22      199.0
23        8.0
24     3200.0
25      305.0
26     1400.0
27     1900.0
28      836.0
29     3700.0
30     8700.0
31     1400.0
32     2400.0
33    11000.0
34     1100.0
35     4500.0
36     1400.0
37     5200.0
38    14000.0
39     8200.0
40     2900.0
41     7700.0
42     2800.0
43     2300.0
44     2500.0
45     5400.0
46      297.0
47     2900.0
48     1600.0
49     3400.0
Name: Plays, dtype: float64

In [31]:
      
# Top 10 plays
top_played = f"""
        SELECT
            Title,
            Plays
        FROM
            games
        GROUP BY
            Title
        ORDER BY
            Plays desc
        LIMIT 10;
    """


df2 = pd.read_sql(text(top_played), con=engine)
df2.head(11)

,Title,Plays
0,Minecraft,33000.0
1,The Legend of Zelda: Breath of the Wild,30000.0
2,Grand Theft Auto V,30000.0
3,Portal 2,29000.0
4,Undertale,28000.0
5,Portal,28000.0
6,Super Smash Bros. Ultimate,25000.0
7,Super Mario Odyssey,25000.0
8,Among Us,25000.0
9,The Elder Scrolls V: Skyrim,22000.0


In [32]:
      
# Bottom 10 plays
bot_played = f"""
        SELECT
            Title as labels,
            Plays as "values"
        FROM
            games
        GROUP BY
            Title
        ORDER BY
            Plays asc
        LIMIT 10;
    """


df2 = pd.read_sql(text(bot_played), con=engine)
df2.head(11)

,labels,values
0,Elden Ring: Shadow of the Erdtree,1.0
1,Anonymous;Code,8.0
2,Tokyo Necro,14.0
3,Megami Ibunroku Persona,16.0
4,Phantom Brigade,16.0
5,Better Call Saul Kart,26.0
6,Bayonetta Origins: Cereza and the Lost Demon,48.0
7,Garten of Banban 2,55.0
8,Another Pok√©mon Game,59.0
9,Duskers,61.0


In [33]:
      
# Most Common
top_genre = f"""
        SELECT
            Genres,
            count(*) as num_genres
        FROM
            games
        GROUP BY
            Genres
        ORDER BY
            num_genres desc
        LIMIT 10;
    """


df2 = pd.read_sql(text(top_genre), con=engine)
df2.head(11)

,Genres,num_genres
0,Adventure,727
1,RPG,77
2,Shooter,61
3,Arcade,42
4,Fighting,38
5,Indie,37
6,Platform,35
7,Brawler,25
8,Racing,14
9,Simulator,12


In [34]:
# Least Common Genres
bot_genre = f"""
        SELECT
            Genres,
            count(*) as num_genres
        FROM
            games
        GROUP BY
            Genres
        ORDER BY
            num_genres asc
        LIMIT 10;
    """


df2 = pd.read_sql(text(bot_genre), con=engine)
df2.head(11)

,Genres,num_genres
0,Strategy,1
1,None,2
2,Visual Novel,2
3,MOBA,3
4,Point-and-Click,3
5,Real Time Strategy,3
6,Music,6
7,Sport,7
8,Puzzle,10
9,Card & Board Game,12


In [35]:
# least Common Genres(limitation)
bot_genre2 = f"""
        SELECT
            Genres,
            Title,
            count(*) as num_genres
        FROM
            games
        GROUP BY
            Genres,
            Title
        ORDER BY
            num_genres asc
        LIMIT 10;
    """


df2 = pd.read_sql(text(bot_genre2), con=engine)
df2.head(11)

,Genres,Title,num_genres
0,None,Armored Core 2,1
1,None,Peggle Deluxe,1
2,Adventure,13 Sentinels: Aegis Rim,1
3,Adventure,428: Shibuya Scramble,1
4,Adventure,A Hat in Time,1
5,Adventure,A Plague Tale: Innocence,1
6,Adventure,A Short Hike,1
7,Adventure,A Way Out,1
8,Adventure,AI: The Somnium Files - Nirvana Initiative,1
9,Adventure,Abzu,1


In [36]:
# Top 10 rated plays
top_played_rating = f"""
        SELECT
            Title,
            Rating,
            Plays
        FROM
            games
        GROUP BY
            Title
        ORDER BY
            Rating desc
        LIMIT 10;
    """


df2 = pd.read_sql(text(top_played_rating), con=engine)
df2.head(11)

,Title,Rating,Plays
0,Elden Ring: Shadow of the Erdtree,4.8,1.0
1,Umineko: When They Cry Chiru,4.6,1700.0
2,Tokyo Necro,4.6,14.0
3,The Great Ace Attorney 2: Resolve,4.6,1100.0
4,Sekiro: Shadows Die Twice - GOTY Edition,4.6,1400.0
5,Outer Wilds,4.6,7700.0
6,Metal Gear Solid 3: Subsistence,4.6,3700.0
7,Metal Gear Solid 3: Snake Eater HD Edition,4.6,2500.0
8,Hitman World of Assassination,4.6,167.0
9,Half-Life: Alyx,4.6,1900.0


In [37]:
# Top 10 played titles by rating
top_played_plays = f"""
        SELECT
            Title,
            Rating,
            Plays
        FROM
            games
        GROUP BY
            Title
        ORDER BY
            Plays desc
        LIMIT 10;
    """


df2 = pd.read_sql(text(top_played_plays), con=engine)
df2.head(11)

,Title,Rating,Plays
0,Minecraft,4.3,33000.0
1,The Legend of Zelda: Breath of the Wild,4.4,30000.0
2,Grand Theft Auto V,3.8,30000.0
3,Portal 2,4.4,29000.0
4,Undertale,4.2,28000.0
5,Portal,4.1,28000.0
6,Super Smash Bros. Ultimate,4.1,25000.0
7,Super Mario Odyssey,4.2,25000.0
8,Among Us,3.0,25000.0
9,The Elder Scrolls V: Skyrim,3.7,22000.0


In [38]:
# Top 10 rated playing
top_playing_rating = f"""
        SELECT
            Title,
            Rating,
            Playing
        FROM
            games
        GROUP BY
            Title
        ORDER BY
            Playing desc
        LIMIT 10;
    """


df2 = pd.read_sql(text(top_playing_rating), con=engine)
df2.head(11)

,Title,Rating,Playing
0,Elden Ring,4.5,3800.0
1,Hades,4.3,3200.0
2,Genshin Impact,2.6,2700.0
3,The Legend of Zelda: Breath of the Wild,4.4,2500.0
4,Hollow Knight,4.4,2400.0
5,Persona 5 Royal,4.4,2300.0
6,Stardew Valley,4.1,2000.0
7,Yakuza 0,4.4,1800.0
8,Minecraft,4.3,1800.0
9,Animal Crossing: New Horizons,3.6,1800.0


In [39]:
# Top 10 rated playing
bot_rating = f"""
        SELECT
            Title,
            Rating,
            Playing
        FROM
            games
        GROUP BY
            Title
        ORDER BY
            Rating desc
        LIMIT 10;
    """


df2 = pd.read_sql(text(bot_rating), con=engine)
df2.head(11)

,Title,Rating,Playing
0,Elden Ring: Shadow of the Erdtree,4.8,0.0
1,Umineko: When They Cry Chiru,4.6,108.0
2,Tokyo Necro,4.6,22.0
3,The Great Ace Attorney 2: Resolve,4.6,75.0
4,Sekiro: Shadows Die Twice - GOTY Edition,4.6,153.0
5,Outer Wilds,4.6,661.0
6,Metal Gear Solid 3: Subsistence,4.6,50.0
7,Metal Gear Solid 3: Snake Eater HD Edition,4.6,34.0
8,Hitman World of Assassination,4.6,47.0
9,Half-Life: Alyx,4.6,210.0
